In [15]:
# Importing Necessary Libraries

!pip install arff
import numpy as np
import pandas as pd
import arff # This is the ARFF library, which is used to create ARFF files.
import nltk
from collections import Counter
import re

In [16]:
!pip install liac-arff

In [17]:
# Reading the data

file_path = 'english_big.txt'

df = pd.read_csv(file_path, encoding='unicode_escape',error_bad_lines=False, sep = None,header = None,names = ['first','second','third'])
print("Data Reading Successful")

Data Reading Successful


<ipython-input-17-1e07626b46fa>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, encoding='unicode_escape',error_bad_lines=False, sep = None,header = None,names = ['first','second','third'])
<ipython-input-17-1e07626b46fa>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, encoding='unicode_escape',error_bad_lines=False, sep = None,header = None,names = ['first','second','third'])
Skipping line 11: Expected 3 fields in line 11, saw 5
Skipping line 17: Expected 3 fields in line 17, saw 4
Skipping line 21: Expected 3 fields in line 21, saw 5
Skipping line 22: Expected 3 fields in line 22, saw 5
Skipping line 30: Expected 3 fields in line 30, saw 5
Skipping line 35: Expected 3 fields in line 35, saw 8


In [18]:
df.head()

first  \
0  Urgent! call 09061749602 from Landline. Your c...   
1  +449071512431 URGENT! This is the 2nd attempt ...   
2  FREE for 1st week! No1 Nokia tone 4 ur mob eve...   
3  Urgent! call 09066612661 from landline. Your c...   
4  WINNER!! As a valued network customer you have...   

                                              second third  
0  000 cash await collection SAE T&Cs BOX 528 HP2...  spam  
1                                               spam  None  
2                                               spam  None  
3  000 cash await collection SAE T&Cs PO Box 3 WA...  spam  
4                                               spam  None

# Data Preprocessing

In [19]:
df['third'] = df['third'].replace(np.nan,' ')

In [20]:
df['third'].unique()

array(['spam', ' ', 'ham'], dtype=object)

In [21]:
df['Text'] = df['first'] + df['second'] + df['third']
df = df[['Text']]

In [22]:
df.head()

Text
0  Urgent! call 09061749602 from Landline. Your c...
1  +449071512431 URGENT! This is the 2nd attempt ...
2  FREE for 1st week! No1 Nokia tone 4 ur mob eve...
3  Urgent! call 09066612661 from landline. Your c...
4  WINNER!! As a valued network customer you have...

In [23]:
df.shape

(1256, 1)

In [24]:
# Labelling the data based on spam or ham present in the entire text file
df['Label'] = df['Text'].str.contains('spam', case=False, regex=True)
df['Label'] = df['Label'].map({True: 'spam', False: 'ham'})


<ipython-input-24-033f6db844e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Text'].str.contains('spam', case=False, regex=True)
<ipython-input-24-033f6db844e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label'] = df['Label'].map({True: 'spam', False: 'ham'})


In [25]:
df['Text'] = df['Text'].str.replace('spam|ham', '.', case=False, regex=True)

In [26]:
df['Text'].iloc[0]

'Urgent! call 09061749602 from Landline. Your complimentary 4* Tenerife Holiday or £10000 cash await collection SAE T&Cs BOX 528 HP20 1YF 150ppm 18+.'

In [27]:
df.head(3)

Text Label
0  Urgent! call 09061749602 from Landline. Your c...  spam
1  +449071512431 URGENT! This is the 2nd attempt ...  spam
2  FREE for 1st week! No1 Nokia tone 4 ur mob eve...  spam

In [28]:
# Creating the four different features as described in the project file
df['Character_Count'] = df['Text'].apply(len)

currency_symbols = re.compile(r'[$£€¥]')
df['Currency_Symbol_Count'] = df['Text'].apply(lambda x: len(re.findall(currency_symbols, x)))

df['Numeric_String_Count'] = df['Text'].apply(lambda x: len(re.findall(r'\d+', x)))

nltk.download('punkt')
df['Most_Popular_Word_Count'] = df['Text'].apply(lambda x: Counter(x.split()).most_common(1)[0][1])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [29]:
df.head(3)

Text Label  Character_Count  \
0  Urgent! call 09061749602 from Landline. Your c...  spam              148   
1  +449071512431 URGENT! This is the 2nd attempt ...  spam              163   
2  FREE for 1st week! No1 Nokia tone 4 ur mob eve...  spam              160   

   Currency_Symbol_Count  Numeric_String_Count  Most_Popular_Word_Count  
0                      1                     8                        1  
1                      2                    12                        1  
2                      0                     8                        2

In [30]:
df['Label'].value_counts()

ham     979
spam    277
Name: Label, dtype: int64

In [31]:
df.drop("Text",inplace = True,axis = 1)

In [32]:
df.columns

Index(['Label', 'Character_Count', 'Currency_Symbol_Count',
       'Numeric_String_Count', 'Most_Popular_Word_Count'],
      dtype='object')

In [33]:
df

Label  Character_Count  Currency_Symbol_Count  Numeric_String_Count  \
0     spam              148                      1                     8   
1     spam              163                      2                    12   
2     spam              160                      0                     8   
3     spam              167                      1                     8   
4     spam              159                      1                     4   
...    ...              ...                    ...                   ...   
1251  spam              150                      0                     8   
1252  spam              120                      2                     3   
1253  spam              117                      2                     2   
1254  spam               39                      0                     4   
1255  spam              111                      0                     2   

      Most_Popular_Word_Count  
0                           1  
1                           1  
2                           2  
3                           1  
4                           1  
...                       ...  
1251                        2  
1252                        2  
1253                        2  
1254                        1  
1255                        2  

[1256 rows x 5 columns]

In [34]:
df['Label'].value_counts()

ham     979
spam    277
Name: Label, dtype: int64

In [35]:
df

Label  Character_Count  Currency_Symbol_Count  Numeric_String_Count  \
0     spam              148                      1                     8   
1     spam              163                      2                    12   
2     spam              160                      0                     8   
3     spam              167                      1                     8   
4     spam              159                      1                     4   
...    ...              ...                    ...                   ...   
1251  spam              150                      0                     8   
1252  spam              120                      2                     3   
1253  spam              117                      2                     2   
1254  spam               39                      0                     4   
1255  spam              111                      0                     2   

      Most_Popular_Word_Count  
0                           1  
1                           1  
2                           2  
3                           1  
4                           1  
...                       ...  
1251                        2  
1252                        2  
1253                        2  
1254                        1  
1255                        2  

[1256 rows x 5 columns]

In [36]:
df.columns

Index(['Label', 'Character_Count', 'Currency_Symbol_Count',
       'Numeric_String_Count', 'Most_Popular_Word_Count'],
      dtype='object')

In [37]:
df.columns

Index(['Label', 'Character_Count', 'Currency_Symbol_Count',
       'Numeric_String_Count', 'Most_Popular_Word_Count'],
      dtype='object')

In [38]:
df.to_csv('sms_features.CSV')

In [39]:
df

Label  Character_Count  Currency_Symbol_Count  Numeric_String_Count  \
0     spam              148                      1                     8   
1     spam              163                      2                    12   
2     spam              160                      0                     8   
3     spam              167                      1                     8   
4     spam              159                      1                     4   
...    ...              ...                    ...                   ...   
1251  spam              150                      0                     8   
1252  spam              120                      2                     3   
1253  spam              117                      2                     2   
1254  spam               39                      0                     4   
1255  spam              111                      0                     2   

      Most_Popular_Word_Count  
0                           1  
1                           1  
2                           2  
3                           1  
4                           1  
...                       ...  
1251                        2  
1252                        2  
1253                        2  
1254                        1  
1255                        2  

[1256 rows x 5 columns]

In [40]:
df.columns

Index(['Label', 'Character_Count', 'Currency_Symbol_Count',
       'Numeric_String_Count', 'Most_Popular_Word_Count'],
      dtype='object')

In [41]:
df = df[['Character_Count', 'Currency_Symbol_Count','Numeric_String_Count', 'Most_Popular_Word_Count','Label']]

In [42]:
df.to_csv('SMS_Features.csv',index = False)

In [44]:
df.head(5)

Character_Count  Currency_Symbol_Count  Numeric_String_Count  \
0              148                      1                     8   
1              163                      2                    12   
2              160                      0                     8   
3              167                      1                     8   
4              159                      1                     4   

   Most_Popular_Word_Count Label  
0                        1  spam  
1                        1  spam  
2                        2  spam  
3                        1  spam  
4                        1  spam

from matplotlib import pyplot as plt
_df_0['Character_Count'].plot(kind='hist', bins=20, title='Character_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Currency_Symbol_Count'].plot(kind='hist', bins=20, title='Currency_Symbol_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Numeric_String_Count'].plot(kind='hist', bins=20, title='Numeric_String_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Most_Popular_Word_Count'].plot(kind='hist', bins=20, title='Most_Popular_Word_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_6.plot(kind='scatter', x='Numeric_String_Count', y='Most_Popular_Word_Count', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_7['Character_Count'].plot(kind='line', figsize=(8, 4), title='Character_Count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_8['Currency_Symbol_Count'].plot(kind='line', figsize=(8, 4), title='Currency_Symbol_Count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['Numeric_String_Count'].plot(kind='line', figsize=(8, 4), title='Numeric_String_Count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['Most_Popular_Word_Count'].plot(kind='line', figsize=(8, 4), title='Most_Popular_Word_Count')
plt.gca().spines[['top', 'right']].set_visible(False)